In [6]:
# ====================================
# OPTA API - EXTRACTOR MA2, MA3 Y MA12 (CON SELECCIÓN DE TEMPORADA)
# Modificado para incluir selección de stage/temporada
# ====================================

import json
import numpy as np
import pandas as pd
import hashlib
import requests
import time
import os
from datetime import datetime

In [7]:
# ====================================
# DICCIONARIO DE EVENT TYPES
# ====================================
EVENT_TYPE_MAPPING = {
    1: "Pass",
    2: "Offside Pass", 
    3: "Take On",
    4: "Foul",
    5: "Out",
    6: "Corner Awarded",
    7: "Tackle",
    8: "Interception",
    10: "Save",
    11: "Claim",
    12: "Clearance",
    13: "Miss",
    14: "Post",
    15: "Attempt Saved",
    16: "Goal",
    17: "Card",
    18: "Player off",
    19: "Player on",
    20: "Player retired",
    21: "Player returns",
    22: "Player becomes goalkeeper",
    23: "Goalkeeper becomes player",
    24: "Condition change",
    25: "Official change",
    27: "Start delay",
    28: "End delay",
    29: "Temporary stop",
    30: "End",
    32: "Start",
    34: "Team set up",
    36: "Player changed Jersey number",
    37: "Collection End",
    38: "Temp Goal",
    39: "Temp Attempt",
    40: "Formation change",
    41: "Punch",
    42: "Good skill",
    43: "Deleted event",
    44: "Aerial",
    45: "Challenge",
    47: "Rescinded card",
    49: "Ball recovery",
    50: "Dispossessed",
    51: "Error",
    52: "Keeper pick-up",
    53: "Cross not claimed",
    54: "Smother",
    55: "Offside provoked",
    56: "Shield ball opp",
    57: "Foul throw-in",
    58: "Penalty faced",
    59: "Keeper Sweeper",
    60: "Chance missed",
    61: "Ball touch",
    63: "Temp Save",
    64: "Resume",
    65: "Contentious referee decision",
    67: "50/50",
    68: "Referee Drop Ball",
    69: "Failed to Block",
    70: "Injury Time Announcement",
    71: "Coach Setup",
    72: "Caught Offside",
    73: "Other Ball Contact",
    74: "Blocked Pass",
    75: "Delayed start",
    76: "Early end",
    79: "Coverage interruption",
    80: "Drop of Ball",
    81: "Obstacle",
    82: "Control",
    83: "Attempted tackle",
    84: "Deleted After Review"
}

In [8]:
# ====================================
# CONFIGURACIÓN
# ====================================

outletApiKey = '10lthl3y5chwn1m0fa4mfg3bqy'
secretKey = '1u3x3eovxa0vh1lwmutbygq8xn'
delay_seconds = 30

In [9]:
# ====================================
# FUNCIONES CORE
# ====================================

def requestHeaders():
    """Función OAuth idéntica a la notebook que funciona"""
    timestamp = int(round(time.time() * 1000))
    
    post_url = f'https://oauth.performgroup.com/oauth/token/{outletApiKey}?_fmt=json&_rt=b'
    
    # generate a unique hash
    key = str.encode(outletApiKey + str(timestamp) + secretKey)
    unique_hash = hashlib.sha512(key).hexdigest()
    
    # call the OAuth API (post)
    oauthHeaders = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {unique_hash}',
        'Timestamp': str(timestamp)
    }
    
    BODY = {
        'grant_type': 'client_credentials',
        'scope': 'b2b-feeds-auth'
    }
    
    response = requests.post(post_url, data=BODY, headers=oauthHeaders)
    access_token = response.json()['access_token']
    oauthHeaders = {'Authorization': f'Bearer {access_token}'}
    return oauthHeaders

def get_available_stages_method1():
    """Método 1: Usar Tournament Calendar básico (sin filtros) y filtrar después"""
    requestParameters = {
        "_fmt": "json",
        "_pgSz": "100",
        "_pgNm": "1",
        "_rt": "b"
    }
    
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/tournamentcalendar/{outletApiKey}/'
    
    try:
        print("   🔄 Intentando con Tournament Calendar básico...")
        response = requests.get(sdapi_get_url, headers=requestHeaders(), params=requestParameters)
        response.raise_for_status()
        data = response.json()
        
        stages = {}
        tournament_calendars = data.get('tournamentCalendar', [])
        
        for tc in tournament_calendars:
            stage = tc.get('stage', {})
            competition = tc.get('competition', {})
            stage_id = stage.get('id')
            stage_name = stage.get('name', 'N/A')
            comp_name = competition.get('name', 'N/A')
            comp_id = competition.get('id', 'N/A')
            
            if stage_id and stage_id not in stages:
                stages[stage_id] = {
                    'name': stage_name,
                    'competition': comp_name,
                    'competition_id': comp_id,
                    'start_date': stage.get('startDate', 'N/A'),
                    'end_date': stage.get('endDate', 'N/A')
                }
        
        return stages
    except Exception as e:
        print(f"   ❌ Error con Tournament Calendar: {e}")
        return {}

def get_available_stages_method2():
    """Método 2: Usar MA1 básico (sin filtros problemáticos) para obtener stages"""
    requestParameters = {
        "_fmt": "json",
        "_pgSz": "100",
        "_pgNm": "1",
        "_rt": "b"
    }
    
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/match/{outletApiKey}/'
    
    try:
        print("   🔄 Intentando con MA1 básico...")
        response = requests.get(sdapi_get_url, headers=requestHeaders(), params=requestParameters)
        response.raise_for_status()
        data = response.json()
        matches = data.get('match', [])
        
        stages = {}
        for match in matches:
            match_info = match.get('matchInfo', {})
            competition = match_info.get('competition', {})
            stage = match_info.get('stage', {})
            stage_id = stage.get('id')
            stage_name = stage.get('name', 'N/A')
            comp_name = competition.get('name', 'N/A')
            comp_id = competition.get('id', 'N/A')
            
            if stage_id and stage_id not in stages:
                stages[stage_id] = {
                    'name': stage_name,
                    'competition': comp_name,
                    'competition_id': comp_id,
                    'start_date': stage.get('startDate', 'N/A'),
                    'end_date': stage.get('endDate', 'N/A')
                }
        
        return stages
    except Exception as e:
        print(f"   ❌ Error con MA1 básico: {e}")
        return {}

def get_known_stages():
    """Método 3: Stages conocidos como fallback"""
    known_stages = {
        # La Liga 2024-25
        '4xu8dwf3cotp5qu0ddi50wkyc': {
            'name': 'La Liga 2024-25',
            'competition': 'La Liga',
            'start_date': '2024-08-01',
            'end_date': '2025-05-31'
        },
        # Premier League 2024-25 (ejemplo)
        '2kwbbcootiqqgmrzs6o5inle5': {
            'name': 'Premier League 2024-25',
            'competition': 'Premier League',
            'start_date': '2024-08-01',
            'end_date': '2025-05-31'
        }
    }

def get_existing_match_ids():
    """Lee todos los parquet existentes y obtiene los Match IDs ya procesados"""
    folder = "datos_opta_parquet"
    existing_match_ids = set()
    
    # Lista de archivos parquet a revisar
    parquet_files = [
        'player_stats.parquet',
        'team_stats.parquet', 
        'player_xg_stats.parquet',
        'xg_events.parquet',
        'abp_events.parquet',
        'team_officials.parquet'
    ]
    
    print("🔍 Revisando archivos existentes...")
    
    for filename in parquet_files:
        filepath = f"{folder}/{filename}"
        if os.path.exists(filepath):
            try:
                df = pd.read_parquet(filepath)
                if not df.empty and 'Match ID' in df.columns:
                    file_match_ids = set(df['Match ID'].unique())
                    existing_match_ids.update(file_match_ids)
                    print(f"   📄 {filename}: {len(file_match_ids)} partidos únicos")
                else:
                    print(f"   📄 {filename}: archivo vacío o sin columna Match ID")
            except Exception as e:
                print(f"   ❌ Error leyendo {filename}: {e}")
        else:
            print(f"   📄 {filename}: no existe")
    
    if existing_match_ids:
        print(f"   ✅ Total Match IDs existentes: {len(existing_match_ids)}")
    else:
        print(f"   📁 No se encontraron datos previos - descarga completa")
    
    return existing_match_ids

def filter_new_matches(matches_df, existing_match_ids):
    """Filtra el DataFrame de partidos para quedarse solo con los nuevos"""
    if matches_df.empty:
        return matches_df
    
    if not existing_match_ids:
        print(f"   🆕 Todos los partidos son nuevos: {len(matches_df)}")
        return matches_df
    
    # Filtrar partidos que NO están en existing_match_ids
    new_matches = matches_df[~matches_df['Match ID'].isin(existing_match_ids)].copy()
    
    total_matches = len(matches_df)
    new_matches_count = len(new_matches)
    existing_matches_count = total_matches - new_matches_count
    
    print(f"   📊 Total partidos encontrados: {total_matches}")
    print(f"   ✅ Ya procesados: {existing_matches_count}")
    print(f"   🆕 Nuevos por procesar: {new_matches_count}")
    
    if new_matches_count == 0:
        print(f"   🎉 ¡Todos los partidos ya están procesados!")
    
    return new_matches

def debug_api_calls():
    """Función de debugging para probar diferentes llamadas a la API"""
    print("🔧 MODO DEBUG - PROBANDO LLAMADAS A LA API")
    
    tests = [
        {
            'name': 'MA1 Básico (sin filtros)',
            'url': f'https://api.performfeeds.com/soccerdata/match/{outletApiKey}/',
            'params': {"_fmt": "json", "_pgSz": "5", "_rt": "b"}
        },
        {
            'name': 'MA1 con Competition Filter',
            'url': f'https://api.performfeeds.com/soccerdata/match/{outletApiKey}/',
            'params': {"_fmt": "json", "_pgSz": "5", "_rt": "b", "comp": "15"}
        },
        {
            'name': 'Tournament Calendar (OT2)',
            'url': f'https://api.performfeeds.com/soccerdata/tournamentcalendar/{outletApiKey}/',
            'params': {"_fmt": "json", "_pgSz": "5", "_rt": "b"}
        },
        {
            'name': 'OT2 con Competition Filter',
            'url': f'https://api.performfeeds.com/soccerdata/tournamentcalendar/{outletApiKey}/',
            'params': {"_fmt": "json", "_pgSz": "5", "_rt": "b", "comp": "15"}
        }
    ]
    
    for test in tests:
        print(f"\n🧪 Probando: {test['name']}")
        try:
            response = requests.get(test['url'], headers=requestHeaders(), params=test['params'])
            print(f"   Status: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                print(f"   ✅ Éxito - Datos obtenidos")
                
                # Mostrar estructura básica
                if 'match' in data:
                    matches = data.get('match', [])
                    print(f"   📊 Partidos encontrados: {len(matches)}")
                    if matches:
                        first_match = matches[0]
                        match_info = first_match.get('matchInfo', {})
                        stage = match_info.get('stage', {})
                        comp = match_info.get('competition', {})
                        print(f"   🏆 Ejemplo - Comp: {comp.get('name', 'N/A')}, Stage: {stage.get('name', 'N/A')}")
                
                elif 'tournamentCalendar' in data:
                    tournaments = data.get('tournamentCalendar', [])
                    print(f"   📅 Tournament Calendars encontrados: {len(tournaments)}")
                    if tournaments:
                        first_tc = tournaments[0]
                        stage = first_tc.get('stage', {})
                        comp = first_tc.get('competition', {})
                        print(f"   🏆 Ejemplo - Comp: {comp.get('name', 'N/A')}, Stage: {stage.get('name', 'N/A')}")
                        
            else:
                print(f"   ❌ Error {response.status_code}: {response.text[:200]}...")
                
        except Exception as e:
            print(f"   💥 Excepción: {str(e)[:200]}...")
    
    print(f"\n🔧 ¿Quieres probar alguna llamada específica? (s/n): ", end="")

def process_matchTeamStats_data(match_id):
    """MA2 Team Stats - MODIFICADO para incluir Team Name"""
    # API Parameters
    requestParameters = {
        "_fmt": "json",
        "detailed": "yes",
        "fx": match_id,
        "_rt": "b"
    }
    
    # GET API
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/matchstats/{outletApiKey}/'
    response = requests.get(
        sdapi_get_url,
        headers=requestHeaders(),
        params=requestParameters
    )
    
    # Check response Status
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print(response.text)
        return pd.DataFrame()
    
    # Extract data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    line_ups = live_data.get('lineUp', [])
    
    # AÑADIR: Obtener información home/away
    home_away_info = get_home_away_info(match_info, live_data)

    
    # Create a DataFrame for team stats
    team_stats_data = []
    
    # Extract team stats
    for team_stats in line_ups:
        team_id = team_stats['contestantId']
        team_name = home_away_info['team_mapping'].get(team_id, {}).get('name', 'N/A')
        team_position = home_away_info['team_mapping'].get(team_id, {}).get('position', 'N/A')

        # AÑADIR: Determinar si es home o away
        is_home = team_id == home_away_info['home_team_id']
        is_away = team_id == home_away_info['away_team_id']
        
        for stat in team_stats['stat']:
            stat_info = {
                'Match ID': match_info.get('id', 'N/A'),
                'Competition ID': competition_info.get('id', 'N/A'),
                'Competition Name': competition_info.get('name', 'N/A'),
                'Week': match_info.get('week', 'N/A'),
                'Stage ID': stage_info.get('id', 'N/A'),
                'Stage Name': stage_info.get('name', 'N/A'),
                'Team ID': team_id,
                'Team Name': team_name,
                'Team Position': team_position,  # AÑADIR
                'Is Home': is_home,  # AÑADIR
                'Is Away': is_away,  # AÑADIR
                'HT Home Score': home_away_info['ht_home'],  # AÑADIR
                'HT Away Score': home_away_info['ht_away'],  # AÑADIR
                'FT Home Score': home_away_info['ft_home'],  # AÑADIR
                'FT Away Score': home_away_info['ft_away'],  # AÑADIR
                'Stat Type': stat.get('type', 'N/A'),
                'Total': stat.get('value', 0)
            }
            team_stats_data.append(stat_info)
    
    # Create a DataFrame and converts "NAN" to 0 value
    df_team_stats = pd.DataFrame(team_stats_data)
    if not df_team_stats.empty:
        df_team_stats = df_team_stats.pivot(
            index=['Team ID', 'Team Name', 'Team Position', 'Is Home', 'Is Away', 
            'HT Home Score', 'HT Away Score', 'FT Home Score', 'FT Away Score',
            'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name'],
            columns='Stat Type', 
            values='Total'
        ).reset_index()
        df_team_stats = df_team_stats.fillna(0)
        
        # Convert all stat columns (excluding metadata) to floats
        metadata_cols = ['Team ID', 'Team Name', 'Team Position', 'Is Home', 'Is Away', 
                        'HT Home Score', 'HT Away Score', 'FT Home Score', 'FT Away Score',
                        'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name']
        stat_cols = [col for col in df_team_stats.columns if col not in metadata_cols]
        for col in stat_cols:
            df_team_stats[col] = pd.to_numeric(df_team_stats[col], errors='coerce').fillna(0).astype(float)
    
    return df_team_stats
    

# Función auxiliar para testing rápido
def test_stage_access():
    """Test rápido para verificar acceso a stages - MEJORADO"""
    print("🔍 TEST RÁPIDO DE ACCESO A STAGES")
    
    # Test 1: Usar el stage ID que sabemos que funciona
    known_stage = "4xu8dwf3cotp5qu0ddi50wkyc"
    
    print(f"\n🧪 Probando MA1 con stage conocido: {known_stage[:12]}...")
    
    requestParameters = {
        "_fmt": "json",
        "_pgSz": "5",
        "stg": known_stage,
        "live": "yes",
        "_rt": "b"
    }
    
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/match/{outletApiKey}/'
    
    try:
        response = requests.get(sdapi_get_url, headers=requestHeaders(), params=requestParameters)
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            matches = data.get('match', [])
            print(f"✅ Éxito! Partidos encontrados: {len(matches)}")
            
            if matches:
                # Mostrar info del primer partido
                first_match = matches[0]
                match_info = first_match.get('matchInfo', {})
                stage = match_info.get('stage', {})
                comp = match_info.get('competition', {})
                week = match_info.get('week', 'N/A')
                
                # Verificar status del partido
                live_data = first_match.get('liveData', {})
                match_details = live_data.get('matchDetails', {})
                match_status = match_details.get('matchStatus', 'N/A')
                
                print(f"📊 Ejemplo partido:")
                print(f"   🏆 Competición: {comp.get('name', 'N/A')}")
                print(f"   🗓️ Stage: {stage.get('name', 'N/A')}")
                print(f"   📅 Jornada: {week}")
                print(f"   ⚽ Status: {match_status}")
                
                # Test adicional: probar con jornada específica
                print(f"\n🧪 Probando MA1 con stage + jornada específica...")
                test_week_params = requestParameters.copy()
                test_week_params["week"] = "1"
                
                response2 = requests.get(sdapi_get_url, headers=requestHeaders(), params=test_week_params)
                if response2.status_code == 200:
                    data2 = response2.json()
                    matches2 = data2.get('match', [])
                    print(f"   ✅ Jornada 1: {len(matches2)} partidos encontrados")
                else:
                    print(f"   ❌ Error con jornada específica: {response2.status_code}")
                
                return True
        else:
            print(f"❌ Error {response.status_code}: {response.text[:200]}")
            
    except Exception as e:
        print(f"💥 Error: {e}")
    
    return False

def get_available_stages(competition_id):
    """Obtiene stages usando múltiples métodos como fallback - ARREGLADO basado en debug"""
    print("🔄 Obteniendo temporadas disponibles...")
    
    # Método 1: Tournament Calendar básico
    stages = get_available_stages_method1()
    if stages:
        print(f"   ✅ Encontradas {len(stages)} temporadas con Tournament Calendar")
        # Filtrar por competición después
        competition_names = {
            '13': 'Premier League',
            '15': ['La Liga', 'Primera División'],  # Múltiples nombres posibles
            '16': 'Serie A',
            '17': 'Bundesliga',
            '18': 'Ligue 1',
            '19': 'Champions League'
        }
        
        target_comps = competition_names.get(competition_id, [])
        if isinstance(target_comps, str):
            target_comps = [target_comps]
        
        if target_comps:
            filtered_stages = {}
            for stage_id, stage_info in stages.items():
                comp_name = stage_info.get('competition', '').lower()
                for target_comp in target_comps:
                    if target_comp.lower() in comp_name or comp_name in target_comp.lower():
                        filtered_stages[stage_id] = stage_info
                        break
            
            if filtered_stages:
                print(f"   🎯 Filtradas {len(filtered_stages)} temporadas para la competición")
                return filtered_stages
        
        # Si no se puede filtrar, devolver todas
        return stages
    
    # Método 2: MA1 básico
    stages = get_available_stages_method2()
    if stages:
        print(f"   🔍 DEBUG - Todas las temporadas encontradas:")
        for stage_id, stage_info in stages.items():
            print(f"      {stage_id[:12]}... - {stage_info.get('name', 'N/A')} - {stage_info.get('competition', 'N/A')}")
        
        # Filtrar por competición
        competition_names = {
            '13': 'Premier League',
            '15': ['La Liga', 'Primera División'],
            '16': 'Serie A',
            '17': 'Bundesliga',
            '18': 'Ligue 1',
            '19': 'Champions League'
        }
        
        target_comps = competition_names.get(competition_id, [])
        if isinstance(target_comps, str):
            target_comps = [target_comps]
        
        if target_comps:
            filtered_stages = {}
            for stage_id, stage_info in stages.items():
                comp_name = stage_info.get('competition', '').lower()
                for target_comp in target_comps:
                    if target_comp.lower() in comp_name or comp_name in target_comp.lower():
                        filtered_stages[stage_id] = stage_info
                        break
            
            if filtered_stages:
                print(f"   🎯 Filtradas {len(filtered_stages)} temporadas para la competición")
                return filtered_stages
        
        return stages
    
    # Método 3: Stages conocidos
    print("   🔄 Usando stages conocidos como fallback...")
    known_stages = get_known_stages()
    
    # Filtrar por competición
    competition_names = {
        '13': 'Premier League',
        '15': 'La Liga', 
        '16': 'Serie A',
        '17': 'Bundesliga',
        '18': 'Ligue 1',
        '19': 'Champions League'
    }
    target_comp = competition_names.get(competition_id, '')
    if target_comp:
        filtered_stages = {k: v for k, v in known_stages.items() 
                         if target_comp.lower() in v.get('competition', '').lower()}
        if filtered_stages:
            print(f"   ✅ Usando {len(filtered_stages)} temporadas conocidas")
            return filtered_stages
    
    return known_stages

def get_match_ids_advanced(competition_levels="13,15", max_matches=50, specific_week=None, stage_id=None):
    """Obtiene match IDs - ARREGLADO para evitar filtros problemáticos"""
    requestParameters = {
        "_fmt": "json",
        "_pgSz": str(max_matches),
        "_pgNm": "1",
        "live": "yes",
        "_rt": "b"
        # REMOVIDO: "status": "played" - causa error 400
        # REMOVIDO: "cvlv": competition_levels - causa error 403
    }
    
    if specific_week:
        requestParameters["week"] = str(specific_week)
    
    # Usar el parámetro 'stg' según la documentación MA1
    if stage_id:
        requestParameters["stg"] = str(stage_id)

    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/match/{outletApiKey}/'
    
    try:
        response = requests.get(sdapi_get_url, headers=requestHeaders(), params=requestParameters)
        response.raise_for_status()
        data = response.json()
        matches = data.get('match', [])
        
        match_list = []
        for match in matches:
            match_info = match.get('matchInfo', {})
            match_id = match_info.get('id')
            if match_id:
                competition = match_info.get('competition', {})
                stage = match_info.get('stage', {})
                contestants = match_info.get('contestant', [])
                teams = [{'id': c.get('id'), 'name': c.get('name'), 'code': c.get('code')} for c in contestants]
                
                # Verificar que el partido esté finalizado usando liveData
                live_data = match.get('liveData', {})
                match_details = live_data.get('matchDetails', {})
                match_status = match_details.get('matchStatus', '')
                
                # Solo incluir partidos finalizados
                if match_status.lower() in ['played', 'finished', 'ft', 'final']:
                    match_data = {
                        'Match ID': match_id,
                        'Competition': competition.get('name', 'N/A'),
                        'Competition ID': competition.get('id', 'N/A'),
                        'Stage ID': stage.get('id', 'N/A'),
                        'Stage Name': stage.get('name', 'N/A'),
                        'Date': match_info.get('date', 'N/A'),
                        'Week': match_info.get('week', 'N/A'),
                        'Match Status': match_status,
                        'Teams': teams
                    }
                    match_list.append(match_data)
        
        return pd.DataFrame(match_list)
    except Exception as e:
        print(f"Error obteniendo partidos: {e}")
        return pd.DataFrame()

# ====================================
# EXTRACTORES - COPIADOS DE LA NOTEBOOK QUE FUNCIONA
# ====================================

def process_matchPlayerStats_data(match_id):
    """MA2 Player Stats + Team Officials - CORREGIDO COMPLETAMENTE"""
    # API Parameters
    requestParameters = {
        "_fmt": "json",
        "detailed": "yes",
        "fx": match_id,
        "_rt": "b"
    }
    
    # GET API
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/matchstats/{outletApiKey}/'
    response = requests.get(
        sdapi_get_url,
        headers=requestHeaders(),
        params=requestParameters
    )
    
    # Check response Status
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print(response.text)
        return pd.DataFrame(), pd.DataFrame()
    
    # Extract data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    line_ups = live_data.get('lineUp', [])
    
    # AÑADIR: Obtener información home/away
    home_away_info = get_home_away_info(match_info, live_data)
    
    # Create a DataFrame for player stats
    player_stats_data = []
    
    # Extract player stats
    for line_up in line_ups:
        team_id = line_up.get('contestantId')
        team_name = home_away_info['team_mapping'].get(team_id, {}).get('name', 'N/A')
        team_position = home_away_info['team_mapping'].get(team_id, {}).get('position', 'N/A')
                
        # AÑADIR: Determinar si es home o away
        is_home = team_id == home_away_info['home_team_id']
        is_away = team_id == home_away_info['away_team_id']
        
        for player in line_up.get('player', []):
            player_entry = {
                'Match ID': match_info.get('id', 'N/A'),
                'Competition ID': competition_info.get('id', 'N/A'),
                'Competition Name': competition_info.get('name', 'N/A'),
                'Week': match_info.get('week', 'N/A'),
                'Stage ID': stage_info.get('id', 'N/A'),
                'Stage Name': stage_info.get('name', 'N/A'),
                'Team ID': team_id,
                'Team Name': team_name,
                'Team Position': team_position,  # AÑADIR
                'Is Home': is_home,  # AÑADIR
                'Is Away': is_away,  # AÑADIR
                'HT Home Score': home_away_info['ht_home'],  # AÑADIR
                'HT Away Score': home_away_info['ht_away'],  # AÑADIR
                'FT Home Score': home_away_info['ft_home'],  # AÑADIR
                'FT Away Score': home_away_info['ft_away'],  # AÑADIR
                'Player ID': player.get('playerId', 'N/A'),
                'First Name': player.get('firstName', 'N/A'),
                'Last Name': player.get('lastName', 'N/A'),
                'Match Name': player.get('matchName', 'N/A'),
                'Shirt Number': player.get('shirtNumber', 'N/A'),
                'Position': player.get('position', 'N/A'),
                'Position Side': player.get('positionSide', 'N/A'),
                'Formation Place': player.get('formationPlace', 'N/A'),
            }
            
            for stat in player.get('stat', []):
                stat_type = stat.get('type', '')
                stat_value = stat.get('value', 0)
                player_entry[stat_type] = stat_value
            
            player_stats_data.append(player_entry)
    
    # Create a DataFrame for team officials
    team_officials_data = []
    
    # Extract team officials
    for line_up in line_ups:
        team_id = line_up.get('contestantId')
        team_name = home_away_info['team_mapping'].get(team_id, {}).get('name', 'N/A')
        
        for official in line_up.get('teamOfficial', []):
            official_entry = {
                'Match ID': match_info.get('id', 'N/A'),
                'Competition ID': competition_info.get('id', 'N/A'),
                'Competition Name': competition_info.get('name', 'N/A'),
                'Week': match_info.get('week', 'N/A'),
                'Stage ID': stage_info.get('id', 'N/A'),
                'Stage Name': stage_info.get('name', 'N/A'),
                'Team ID': team_id,
                'Team Name': team_name,
                'Official ID': official.get('id', 'N/A'),
                'First Name': official.get('firstName', 'N/A'),
                'Last Name': official.get('lastName', 'N/A'),
                'Short First Name': official.get('shortFirstName', 'N/A'),
                'Short Last Name': official.get('shortlastName', 'N/A'),
                'Known Name': official.get('knownName', 'N/A'),
                'Type': official.get('type', 'N/A')
            }
            team_officials_data.append(official_entry)
    
    # Create DataFrames and convert "NAN" to 0 value
    df_player_stats = pd.DataFrame(player_stats_data).fillna(0)
    df_team_officials = pd.DataFrame(team_officials_data).fillna('N/A')
    
    # Convert all stat columns (excluding metadata) to numeric for player stats
    non_stat_cols = ['Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name', 
                     'Team ID', 'Team Name', 'Team Position', 'Is Home', 'Is Away', 
                     'HT Home Score', 'HT Away Score', 'FT Home Score', 'FT Away Score',
                     'Player ID', 'First Name', 'Last Name', 'Match Name',
                     'Shirt Number', 'Position', 'Position Side', 'Formation Place']
    
    if not df_player_stats.empty:
        stat_cols = [col for col in df_player_stats.columns if col not in non_stat_cols]
        
        for col in stat_cols:
            df_player_stats[col] = pd.to_numeric(df_player_stats[col], errors='coerce').fillna(0)
        
        # Optionally, convert all floats to ints if appropriate
        df_player_stats[stat_cols] = df_player_stats[stat_cols].astype(float)
    
    return df_player_stats, df_team_officials

def process_match_events_data(match_id):
    """MA3 Match Events - CORREGIDO"""
    # API Parameters
    requestParameters = {
        "_fmt": "json",
        "fx": match_id,
        "_rt": "b"
    }
    
    # GET API
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/matchevent/{outletApiKey}/'
    response = requests.get(
        sdapi_get_url,
        headers=requestHeaders(),
        params=requestParameters
    )
    
    # Check response Status
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print(response.text)
        return pd.DataFrame()
    
    # Extract data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    events = live_data.get('event', [])
    
    # AÑADIR: Obtener información home/away
    home_away_info = get_home_away_info(match_info, live_data)
    
    # Find all unique qualifier IDs
    qualifier_ids = set()
    for event in events:
        for q in event.get('qualifier', []):
            qualifier_ids.add(str(q.get('qualifierId', '')))
    
    # Initialize DataFrame columns
    columns = [
        'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name',
        'EventId', 'timeStamp', 'contestantId', 'Team ID', 'Team Name', 'Team Position',
        'Is Home', 'Is Away', 'HT Home Score', 'HT Away Score', 'FT Home Score', 'FT Away Score',
        'periodId', 'timeMin', 'timeSec', 'playerId', 'playerName', 'typeId', 'Event Name', 
        'outcome', 'x', 'y'
    ] + [f'qualifier {qid}' for qid in qualifier_ids]
    
    # Create a DataFrame for events
    events_data = []
    
    for event in events:
        contestant_id = event.get('contestantId', None)
        team_name = home_away_info['team_mapping'].get(contestant_id, {}).get('name', 'N/A') if contestant_id else 'N/A'
        team_position = home_away_info['team_mapping'].get(contestant_id, {}).get('position', 'N/A') if contestant_id else 'N/A'
        
        # AÑADIR: Determinar si es home o away
        is_home = contestant_id == home_away_info['home_team_id']
        is_away = contestant_id == home_away_info['away_team_id']
        
        type_id = event.get('typeId', None)
        event_name = EVENT_TYPE_MAPPING.get(type_id, 'Unknown Event') if type_id else 'Unknown Event'
        
        event_info = {
            'Match ID': match_info.get('id', 'N/A'),
            'Competition ID': competition_info.get('id', 'N/A'),
            'Competition Name': competition_info.get('name', 'N/A'),
            'Week': match_info.get('week', 'N/A'),
            'Stage ID': stage_info.get('id', 'N/A'),
            'Stage Name': stage_info.get('name', 'N/A'),
            'EventId': event.get('eventId', None),
            'typeId': type_id,
            'Event Name': event_name,
            'periodId': event.get('periodId', None),
            'timeMin': event.get('timeMin', None),
            'timeSec': event.get('timeSec', None),
            'contestantId': contestant_id,
            'Team ID': contestant_id,
            'Team Name': team_name,
            'Team Position': team_position,  # AÑADIR
            'Is Home': is_home,  # AÑADIR
            'Is Away': is_away,  # AÑADIR
            'HT Home Score': home_away_info['ht_home'],  # AÑADIR
            'HT Away Score': home_away_info['ht_away'],  # AÑADIR
            'FT Home Score': home_away_info['ft_home'],  # AÑADIR
            'FT Away Score': home_away_info['ft_away'],  # AÑADIR
            'playerId': event.get('playerId', None),
            'playerName': event.get('playerName', None),
            'outcome': event.get('outcome', None),
            'x': event.get('x', None),
            'y': event.get('y', None),
            'timeStamp': event.get('timeStamp', None),
        }
        
        # Initialize all qualifiers to 0
        for qid in qualifier_ids:
            event_info[f'qualifier {qid}'] = 0
        
        # Update with actual qualifier values
        for q in event.get('qualifier', []):
            event_info[f'qualifier {q["qualifierId"]}'] = q.get('value', None)
        
        events_data.append(event_info)
    
    events_df = pd.DataFrame(events_data, columns=columns)
    return events_df

def process_xG_matchPlayerStats_data(match_id):
    """MA12 Player xG Stats + Team Officials - MODIFICADO para incluir teamOfficial"""
    # API Parameters
    requestParameters = {
        "_fmt": "json",
        "fx": match_id,
        "_rt": "b"
    }
    
    # GET API
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{outletApiKey}/'
    response = requests.get(
        sdapi_get_url,
        headers=requestHeaders(),
        params=requestParameters
    )
    
    # Create DataFrames for xG player stats and team officials
    player_xGstats_data = []
    team_officials_data = []
    
    # Check response Status
    if response.status_code == 200:
        data = response.json()
        
        # Extract data
        match_info = data.get('matchInfo', {})
        competition_info = match_info.get('competition', {})
        stage_info = match_info.get('stage', {})
        live_data = data.get('liveData', {})
        line_ups = live_data.get('lineUp', [])
        
        # AÑADIR: Obtener información home/away
        home_away_info = get_home_away_info(match_info, live_data)
        
        # Extract player stats
        for line_up in line_ups:
            team_id = line_up.get('contestantId')
            team_name = home_away_info['team_mapping'].get(team_id, {}).get('name', 'N/A')            
            for player in line_up.get('player', []):
                player_entry = {
                    'Match ID': match_info.get('id', 'N/A'),
                    'Competition ID': competition_info.get('id', 'N/A'),
                    'Competition Name': competition_info.get('name', 'N/A'),
                    'Week': match_info.get('week', 'N/A'),
                    'Stage ID': stage_info.get('id', 'N/A'),
                    'Stage Name': stage_info.get('name', 'N/A'),
                    'Team ID': team_id,
                    'Team Name': team_name,
                    'Player ID': player.get('playerId', 'N/A'),
                    'First Name': player.get('firstName', 'N/A'),
                    'Last Name': player.get('lastName', 'N/A'),
                    'Match Name': player.get('matchName', 'N/A'),
                    'Shirt Number': player.get('shirtNumber', 'N/A'),
                    'Position': player.get('position', 'N/A'),
                    'Position Side': player.get('positionSide', 'N/A'),
                    'Formation Place': player.get('formationPlace', 'N/A'),
                }
                
                for stat in player.get('stat', []):
                    stat_type = stat.get('type', '')
                    stat_value = stat.get('value', 0)
                    player_entry[stat_type] = stat_value
                
                player_xGstats_data.append(player_entry)
            
            # Extract team officials
            for official in line_up.get('teamOfficial', []):
                official_entry = {
                    'Match ID': match_info.get('id', 'N/A'),
                    'Competition ID': competition_info.get('id', 'N/A'),
                    'Competition Name': competition_info.get('name', 'N/A'),
                    'Week': match_info.get('week', 'N/A'),
                    'Stage ID': stage_info.get('id', 'N/A'),
                    'Stage Name': stage_info.get('name', 'N/A'),
                    'Team ID': team_id,
                    'Team Name': team_name,  # En MA12 firstName contiene el nombre del equipo según tu comentario
                    'Official ID': official.get('id', 'N/A'),
                    'First Name': official.get('firstName', 'N/A'),
                    'Last Name': official.get('lastName', 'N/A'),
                    'Short First Name': official.get('shortFirstName', 'N/A'),
                    'Short Last Name': official.get('shortlastName', 'N/A'),
                    'Known Name': official.get('knownName', 'N/A'),
                    'Type': official.get('type', 'N/A')
                }
                team_officials_data.append(official_entry)
    
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print(response.text)
        return pd.DataFrame(), pd.DataFrame()
    
    # Create DataFrames and convert "NAN" to 0 value
    df_player_xGstats = pd.DataFrame(player_xGstats_data).fillna(0)
    df_team_officials = pd.DataFrame(team_officials_data).fillna('N/A')
    
    # Convert all stat columns (excluding metadata) to numeric for player xG stats
    non_stat_cols = ['Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name', 
                     'Team ID', 'Team Name', 'Player ID', 'First Name', 'Last Name', 'Match Name',
                     'Shirt Number', 'Position', 'Position Side', 'Formation Place']
    
    if not df_player_xGstats.empty:
        stat_cols = [col for col in df_player_xGstats.columns if col not in non_stat_cols]
        
        for col in stat_cols:
            df_player_xGstats[col] = pd.to_numeric(df_player_xGstats[col], errors='coerce').fillna(0)
        
        # Optionally, convert all floats to ints if appropriate
        df_player_xGstats[stat_cols] = df_player_xGstats[stat_cols].astype(float)
    
    return df_player_xGstats, df_team_officials

def process_xG_match_events(match_id):
    """MA12 xG Events - CORREGIDO"""
    # API Parameters
    requestParameters = {
        "_rt": "b",
        "_fmt": "json",
        "fx": match_id
    }
    
    # GET API
    sdapi_get_url = f'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{outletApiKey}/'
    
    response = requests.get(
        sdapi_get_url,
        headers=requestHeaders(),
        params=requestParameters
    )
    
    # Check response Status
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        print(response.text)
        return pd.DataFrame()
    
    # Extract event data
    match_info = data.get('matchInfo', {})
    competition_info = match_info.get('competition', {})
    stage_info = match_info.get('stage', {})
    live_data = data.get('liveData', {})
    xG_events = live_data.get('event', [])
    
    # AÑADIR: Obtener información home/away
    home_away_info = get_home_away_info(match_info, live_data)
    
    # Find all unique qualifier IDs
    qualifier_ids = {'321', '322'}
    
    # Initialize DataFrame columns
    columns = [
        'Match ID', 'Competition ID', 'Competition Name', 'Week', 'Stage ID', 'Stage Name', 'EventId', 'timeStamp', 
        'contestantId', 'Team ID', 'Team Name', 'periodId', 'timeMin', 'timeSec',
        'playerId', 'playerName', 'typeId', 'Event Name', 'outcome', 'x', 'y'] + [f'qualifier {qid}' for qid in qualifier_ids]
    
    # Create a DataFrame for xG events
    xG_events_data = []
    
    for event in xG_events:
        contestant_id = event.get('contestantId', None)
        team_name = home_away_info['team_mapping'].get(contestant_id, {}).get('name', 'N/A') if contestant_id else 'N/A'  # CORREGIDO
        type_id = event.get('typeId', None)
        event_name = EVENT_TYPE_MAPPING.get(type_id, 'Unknown Event') if type_id else 'Unknown Event'
        
        xG_event_info = {
            'Match ID': match_info.get('id', 'N/A'),
            'Competition ID': competition_info.get('id', 'N/A'),
            'Competition Name': competition_info.get('name', 'N/A'),
            'Week': match_info.get('week', 'N/A'),
            'Stage ID': stage_info.get('id', 'N/A'),
            'Stage Name': stage_info.get('name', 'N/A'),
            'EventId': event.get('eventId', None),
            'typeId': type_id,
            'Event Name': event_name,
            'periodId': event.get('periodId', None),
            'timeMin': event.get('timeMin', None),
            'timeSec': event.get('timeSec', None),
            'contestantId': contestant_id,
            'Team ID': contestant_id,
            'Team Name': team_name,
            'playerId': event.get('playerId', None),
            'playerName': event.get('playerName', None),
            'outcome': event.get('outcome', None),
            'x': event.get('x', None),
            'y': event.get('y', None),
            'timeStamp': event.get('timeStamp', None),
        }
        
        for qid in qualifier_ids:
            xG_event_info[f'qualifier {qid}'] = 0
        
        # Update with actual qualifier values
        for q in event.get('qualifier', []):
            xG_event_info[f'qualifier {q["qualifierId"]}'] = q.get('value', None)
        
        xG_events_data.append(xG_event_info)
    
    xG_events_df = pd.DataFrame(xG_events_data, columns=columns)
    return xG_events_df

# ====================================
# FUNCIONES PRINCIPALES - MODIFICADAS
# ====================================
def get_current_season_stage():
    """Obtiene el Stage ID de la temporada actual (que no sea 23/24 ni 24/25)"""
    known_stages = ['49d7kwlzobzuyja3x2bzwe3o4', '4xu8dwf3cotp5qu0ddi50wkyc']
    
    # Usar el método que ya funciona para obtener stages
    stages = get_available_stages_method2()
    
    for stage_id, stage_info in stages.items():
        comp_name = stage_info.get('competition', '').lower()
        if 'primera' in comp_name or 'la liga' in comp_name:
            if stage_id not in known_stages:
                return stage_id
    
    return None
    
def interactive_competition_selection():
    """Selección simplificada solo para temporadas de La Liga"""
    print("\n🇪🇸 LA LIGA - SELECCIONA TEMPORADA:")
    
    current_stage = get_current_season_stage()
    
    seasons = {
        '1': {
            'name': '2023/24',
            'stage_id': '49d7kwlzobzuyja3x2bzwe3o4'
        },
        '2': {
            'name': '2024/25', 
            'stage_id': '4xu8dwf3cotp5qu0ddi50wkyc'
        },
        '3': {
            'name': '2025/26',
            'stage_id': current_stage
        }
    }
    
    for key, season in seasons.items():
        status = "✅" if season['stage_id'] else "❌"
        print(f"  {key}: Temporada {season['name']} {status}")
    
    choice = input(f"\n🗓️ Selecciona temporada (1-3): ").strip()
    
    if choice in seasons:
        selected = seasons[choice]
        max_week = int(input(f"\n📅 ¿Hasta qué jornada descargar? (ej: 2): ").strip())
        
        return {
            'competition_id': '15',
            'stage_id': selected['stage_id'],
            'stage_name': f"La Liga {selected['name']}",
            'max_week': max_week
        }
    else:
        print("❌ Selección no válida")
        return None

def get_matches_by_weeks(competition_id, stage_id, max_week):
    """Obtiene partidos por jornadas - ARREGLADO para usar solo stage_id"""
    all_matches = []
    
    for week in range(1, max_week + 1):
        print(f"📅 Jornada {week}...")
        matches_df = get_match_ids_advanced(
            max_matches=50,
            specific_week=str(week),
            stage_id=stage_id  # Solo usamos stage_id, no competition_levels
        )
        
        if not matches_df.empty:
            print(f"   ✅ Encontrados {len(matches_df)} partidos en jornada {week}")
            all_matches.append(matches_df)
        else:
            print(f"   ⚠️ No se encontraron partidos en jornada {week}")
        time.sleep(2)
    
    if all_matches:
        result_df = pd.concat(all_matches, ignore_index=True)
        print(f"\n📊 Total partidos encontrados: {len(result_df)}")
        return result_df
    else:
        return pd.DataFrame()
        
def save_to_parquet(data_dict, filename_prefix=None):
    """Guarda datos en parquet de forma incremental - VERSIÓN DEFINITIVA"""
    folder = "datos_opta_parquet"
    os.makedirs(folder, exist_ok=True)
    
    # Nombres fijos para los archivos y sus claves de duplicación
    file_config = {
        'player_stats': {
            'filename': 'player_stats.parquet',
            'duplicate_keys': ['Match ID', 'Player ID']
        },
        'team_stats': {
            'filename': 'team_stats.parquet', 
            'duplicate_keys': ['Match ID', 'Team ID']
        },
        'player_xg_stats': {
            'filename': 'player_xg_stats.parquet',
            'duplicate_keys': ['Match ID', 'Player ID']
        },
        'xg_events': {
            'filename': 'xg_events.parquet',
            'duplicate_keys': ['Match ID', 'EventId']
        },
        'match_events': {
            'filename': 'abp_events.parquet',
            'duplicate_keys': ['Match ID', 'EventId']
        },
        'team_officials': {
            'filename': 'team_officials.parquet',
            'duplicate_keys': ['Match ID', 'Team ID', 'Official ID']
        }
    }
    
    for data_type, df in data_dict.items():
        if not df.empty and data_type in file_config:
            config = file_config[data_type]
            filename = f"{folder}/{config['filename']}"
            duplicate_keys = config['duplicate_keys']
            
            print(f"🔄 Procesando {data_type}...")
            
            try:
                # PASO 1: Crear una copia para no modificar el original
                df_copy = df.copy()
                
                # PASO 2: Conversión ROBUSTA de tipos de datos
                print(f"   📝 Limpiando tipos de datos...")
                
                # Convertir TODAS las columnas qualifier a string sin excepciones
                qualifier_cols = [col for col in df_copy.columns if col.startswith('qualifier')]
                for col in qualifier_cols:
                    try:
                        # Forzar conversión a string manejando todos los casos
                        df_copy[col] = df_copy[col].fillna('0').astype(str)
                        # Limpiar valores problemáticos
                        df_copy[col] = df_copy[col].replace(['nan', 'None', 'null', ''], '0')
                    except Exception as e:
                        print(f"   ⚠️ Error en columna {col}: {e}")
                        df_copy[col] = '0'  # Valor por defecto
                
                # Convertir columnas booleanas a string
                bool_cols = [col for col in df_copy.columns if col in ['Is Home', 'Is Away']]
                for col in bool_cols:
                    try:
                        df_copy[col] = df_copy[col].astype(str)
                    except:
                        df_copy[col] = 'False'
                
                # Convertir columnas numéricas problemáticas
                numeric_cols = [col for col in df_copy.columns if col in ['timeMin', 'timeSec', 'x', 'y', 'eventId', 'EventId', 'periodId']]
                for col in numeric_cols:
                    try:
                        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
                    except:
                        df_copy[col] = None
                
                # Limpiar TODAS las columnas object
                for col in df_copy.columns:
                    if df_copy[col].dtype == 'object':
                        try:
                            df_copy[col] = df_copy[col].fillna('N/A').astype(str)
                            df_copy[col] = df_copy[col].replace(['nan', 'None', 'null'], 'N/A')
                        except:
                            df_copy[col] = 'N/A'
                
                # PASO 3: Intentar guardar el archivo
                if os.path.exists(filename):
                    existing_df = pd.read_parquet(filename)
                    
                    # Verificar claves de duplicación
                    available_keys = [key for key in duplicate_keys if key in df_copy.columns and key in existing_df.columns]
                    
                    if available_keys:
                        combined_df = pd.concat([existing_df, df_copy], ignore_index=True)
                        combined_df = combined_df.drop_duplicates(subset=available_keys, keep='last')
                        print(f"💾 Actualizado: {filename} ({len(existing_df)} → {len(combined_df)} filas)")
                        combined_df.to_parquet(filename, index=False)
                    else:
                        combined_df = pd.concat([existing_df, df_copy], ignore_index=True)
                        print(f"💾 Actualizado (sin deduplicación): {filename} ({len(existing_df)} → {len(combined_df)} filas)")
                        combined_df.to_parquet(filename, index=False)
                else:
                    print(f"💾 Creado: {filename} ({len(df_copy)} filas)")
                    df_copy.to_parquet(filename, index=False)
                    
                print(f"   ✅ {data_type} guardado exitosamente")
                    
            except Exception as e:
                print(f"❌ Error con parquet en {data_type}: {e}")
                print(f"   🔄 Intentando guardar como CSV...")
                
                # FALLBACK: Guardar como CSV
                try:
                    csv_filename = filename.replace('.parquet', '.csv')
                    df_copy.to_csv(csv_filename, index=False)
                    print(f"💾 Guardado como CSV: {csv_filename}")
                except Exception as e2:
                    print(f"❌ Error también con CSV: {e2}")
                    
                    # ÚLTIMO RECURSO: Guardar solo metadatos
                    try:
                        metadata_filename = filename.replace('.parquet', '_metadata.txt')
                        with open(metadata_filename, 'w') as f:
                            f.write(f"Data type: {data_type}\n")
                            f.write(f"Rows: {len(df_copy)}\n")
                            f.write(f"Columns: {list(df_copy.columns)}\n")
                            f.write(f"Error: {str(e)}\n")
                        print(f"📝 Metadatos guardados: {metadata_filename}")
                    except:
                        print(f"💥 Error total con {data_type} - omitiendo...")
                        
        elif not df.empty:
            print(f"⚠️ Tipo de datos desconocido: {data_type} - omitido")
        else:
            print(f"📄 {data_type}: DataFrame vacío - omitido")

def main_interactive():
    """Función principal - MODIFICADA para descarga incremental"""
    print("🎯 EXTRACCIÓN OPTA - MA2, MA3 Y MA12 CON SELECCIÓN DE TEMPORADA")
    
    config = interactive_competition_selection()
    if not config:
        return None
    
    print(f"\n✅ Configuración:")
    print(f"   🏆 Competición: {config['competition_id']}")
    print(f"   🗓️ Temporada: {config['stage_name']}")
    print(f"   📅 Jornadas: 1-{config['max_week']}")
    print(f"   📊 Feeds: MA2 (Match Stats), MA3 (Events), MA12 (xG)")
    
    # NUEVO: Obtener Match IDs ya procesados
    existing_match_ids = get_existing_match_ids()
    
    print("\n🔄 Obteniendo partidos...")
    all_matches_df = get_matches_by_weeks(config['competition_id'], config['stage_id'], config['max_week'])
    
    if all_matches_df.empty:
        print("❌ No se encontraron partidos")
        print("\n🔧 POSIBLES SOLUCIONES:")
        print("   1. Verificar que el Stage ID sea correcto")
        print("   2. Probar con una jornada específica que sepas que tiene partidos")
        print("   3. Verificar permisos de API")
        return None
    
    # NUEVO: Filtrar solo partidos nuevos
    print("\n🔍 Filtrando partidos nuevos...")
    new_matches_df = filter_new_matches(all_matches_df, existing_match_ids)
    
    if new_matches_df.empty:
        print("🎉 ¡No hay partidos nuevos que procesar!")
        print("💾 Los archivos existentes ya contienen todos los datos solicitados.")
        
        # Mostrar resumen de datos existentes
        print("\n📊 RESUMEN DE DATOS EXISTENTES:")
        folder = "datos_opta_parquet"
        parquet_files = {
            'player_stats': 'player_stats.parquet',
            'team_stats': 'team_stats.parquet', 
            'player_xg_stats': 'player_xg_stats.parquet',
            'xg_events': 'xg_events.parquet',
            'match_events': 'abp_events.parquet',
            'team_officials': 'team_officials.parquet'
        }
        
        result = {}
        for data_type, filename in parquet_files.items():
            filepath = f"{folder}/{filename}"
            if os.path.exists(filepath):
                df = pd.read_parquet(filepath)
                result[data_type] = df
                print(f"   ✅ {data_type}: {len(df)} filas")
            else:
                result[data_type] = pd.DataFrame()
                print(f"   📄 {data_type}: archivo no existe")
        
        return result
    
    print(f"\n📊 Procesando {len(new_matches_df)} partidos nuevos...")
    match_ids = new_matches_df['Match ID'].tolist()
    
    # Procesar datos (solo partidos nuevos)
    all_data = {
        'player_stats': [],
        'team_stats': [],
        'player_xg_stats': [],
        'xg_events': [],
        'match_events': [],
        'team_officials': []
    }
    
    for i, match_id in enumerate(match_ids):
        print(f"⚽ Partido {i+1}/{len(match_ids)}: {match_id}")
        
        # MA2 - Player Stats + Team Officials
        player_stats_df, team_officials_df = process_matchPlayerStats_data(match_id)
        if not player_stats_df.empty:
            all_data['player_stats'].append(player_stats_df)
        if not team_officials_df.empty:
            all_data['team_officials'].append(team_officials_df)
        
        # MA2 - Team Stats
        team_stats_df = process_matchTeamStats_data(match_id)
        if not team_stats_df.empty:
            all_data['team_stats'].append(team_stats_df)
        
        # MA3 - Match Events
        match_events_df = process_match_events_data(match_id)
        if not match_events_df.empty:
            all_data['match_events'].append(match_events_df)
        
        # MA12 - Player xG Stats + Team Officials 
        player_xg_df, team_officials_xg_df = process_xG_matchPlayerStats_data(match_id)
        if not player_xg_df.empty:
            all_data['player_xg_stats'].append(player_xg_df)
        if not team_officials_xg_df.empty:
            all_data['team_officials'].append(team_officials_xg_df)
        
        # MA12 - xG Events
        xg_events_df = process_xG_match_events(match_id)
        if not xg_events_df.empty:
            all_data['xg_events'].append(xg_events_df)
        
        if i < len(match_ids) - 1:
            time.sleep(delay_seconds)
    
    # Combinar DataFrames de nuevos datos
    new_data = {}
    for data_type, df_list in all_data.items():
        if df_list:
            new_data[data_type] = pd.concat(df_list, ignore_index=True)
            print(f"✅ {data_type} (nuevos): {len(new_data[data_type])} filas")
        else:
            new_data[data_type] = pd.DataFrame()
    
    # Guardar (la función save_to_parquet ya maneja la combinación con datos existentes)
    filename_prefix = f"OPTA_Comp{config['competition_id']}_Stage{config['stage_id'][:8]}_Week{config['max_week']}"
    
    print(f"\n💾 Guardando datos nuevos y combinando con existentes...")
    save_to_parquet(new_data)
    
    # Cargar datos finales combinados para retornar
    print(f"\n📊 CARGANDO DATOS FINALES COMBINADOS:")
    folder = "datos_opta_parquet"
    parquet_files = {
        'player_stats': 'player_stats.parquet',
        'team_stats': 'team_stats.parquet', 
        'player_xg_stats': 'player_xg_stats.parquet',
        'xg_events': 'xg_events.parquet',
        'match_events': 'abp_events.parquet',
        'team_officials': 'team_officials.parquet'
    }
    
    final_result = {}
    for data_type, filename in parquet_files.items():
        filepath = f"{folder}/{filename}"
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            final_result[data_type] = df
            print(f"   ✅ {data_type}: {len(df)} filas totales")
        else:
            final_result[data_type] = pd.DataFrame()
    
    print(f"\n✅ Completado! Archivos actualizados con prefijo: {filename_prefix}")
    print(f"🎉 Descarga incremental: {len(new_matches_df)} partidos nuevos procesados")
    
    return final_result

def quick_analysis(data):
    """Análisis rápido"""
    print("\n📊 ANÁLISIS RÁPIDO:")
    
    for data_type, df in data.items():
        if not df.empty:
            print(f"\n{data_type}: {len(df)} filas")
            
            if data_type == 'xg_events':
                # Analizar eventos xG
                xg_events = df[df['qualifier 321'].notna() & (df['qualifier 321'] != '0')]
                if not xg_events.empty:
                    print(f"  🎯 Eventos con xG: {len(xg_events)}")
            
            elif data_type == 'player_stats':
                print(f"  👥 Jugadores únicos: {df['Player ID'].nunique()}")
                print(f"  📊 Partidos únicos: {df['Match ID'].nunique()}")
                print(f"  🗓️ Stages únicos: {df['Stage ID'].nunique()}")
            
            elif data_type == 'team_stats':
                print(f"  🏆 Equipos únicos: {df['Team ID'].nunique()}")
                if 'Is Home' in df.columns:
                    home_teams = df[df['Is Home'] == 'True']['Team Name'].nunique()
                    away_teams = df[df['Is Away'] == 'True']['Team Name'].nunique()
                    print(f"  🏠 Equipos locales: {home_teams}")
                    print(f"  🚌 Equipos visitantes: {away_teams}")
                    
                    # Mostrar marcadores si están disponibles
                    if 'FT Home Score' in df.columns:
                        matches_with_scores = df[df['FT Home Score'].notna()]
                        if not matches_with_scores.empty:
                            print(f"  ⚽ Partidos con marcador final: {len(matches_with_scores)}")

def get_home_away_info(match_info, live_data):
    """Extrae información de home/away del partido"""
    contestants = match_info.get('contestant', [])
    match_details = live_data.get('matchDetails', {})
    
    # Crear mapeo de equipos
    team_mapping = {}
    home_team_id = None
    away_team_id = None
    
    for contestant in contestants:
        team_id = contestant.get('id')
        team_name = contestant.get('name', 'N/A')  # ✅ ESTA LÍNEA ES CORRECTA
        position = contestant.get('position', '')
        
        team_mapping[team_id] = {
            'name': team_name,
            'position': position
        }
        
        if position.lower() == 'home':
            home_team_id = team_id
        elif position.lower() == 'away':
            away_team_id = team_id
    
    # Obtener marcadores si están disponibles
    scores = match_details.get('scores', {})
    ht_scores = scores.get('ht', {})
    ft_scores = scores.get('ft', {})
    
    return {
        'home_team_id': home_team_id,
        'away_team_id': away_team_id,
        'team_mapping': team_mapping,
        'ht_home': ht_scores.get('home', None),
        'ht_away': ht_scores.get('away', None),
        'ft_home': ft_scores.get('home', None),
        'ft_away': ft_scores.get('away', None)
    }

def discover_stage_ids_for_competition(competition_id):
    """Descubre Stage IDs para una competición específica"""
    print(f"🔍 Descubriendo Stage IDs para competición {competition_id}...")
    
    stages = get_available_stages(competition_id)
    print(f"📊 Stages encontrados:")
    for stage_id, stage_info in stages.items():
        if isinstance(stage_info, dict):
            print(f"  '{stage_id}': '{stage_info.get('name', 'N/A')}'")
        else:
            print(f"  '{stage_id}': '{stage_info}'")
    
    return stages

In [10]:
# ====================================
# EJECUCIÓN
# ====================================

if __name__ == "__main__":
    print("🎯 OPTA API - EXTRACTOR CON SELECCIÓN DE TEMPORADA")
    print("📊 Feeds: MA2 (Stats), MA3 (Events), MA12 (xG)")
    
    print(f"\n💡 STAGE ID CONOCIDO:")
    print(f"   🇪🇸 La Liga 2024-25: 4xu8dwf3cotp5qu0ddi50wkyc")
    
    # Ejecutar extracción directa
    data = main_interactive()
    if data:
        quick_analysis(data)

🎯 OPTA API - EXTRACTOR CON SELECCIÓN DE TEMPORADA
📊 Feeds: MA2 (Stats), MA3 (Events), MA12 (xG)

💡 STAGE ID CONOCIDO:
   🇪🇸 La Liga 2024-25: 4xu8dwf3cotp5qu0ddi50wkyc
🎯 EXTRACCIÓN OPTA - MA2, MA3 Y MA12 CON SELECCIÓN DE TEMPORADA

🇪🇸 LA LIGA - SELECCIONA TEMPORADA:
   🔄 Intentando con MA1 básico...
  1: Temporada 2023/24 ✅
  2: Temporada 2024/25 ✅
  3: Temporada 2025/26 ✅

✅ Configuración:
   🏆 Competición: 15
   🗓️ Temporada: La Liga 2024/25
   📅 Jornadas: 1-10
   📊 Feeds: MA2 (Match Stats), MA3 (Events), MA12 (xG)
🔍 Revisando archivos existentes...
   📄 player_stats.parquet: no existe
   📄 team_stats.parquet: no existe
   📄 player_xg_stats.parquet: no existe
   📄 xg_events.parquet: no existe
   📄 abp_events.parquet: no existe
   📄 team_officials.parquet: no existe
   📁 No se encontraron datos previos - descarga completa

🔄 Obteniendo partidos...
📅 Jornada 1...
   ✅ Encontrados 10 partidos en jornada 1
📅 Jornada 2...
   ✅ Encontrados 10 partidos en jornada 2
📅 Jornada 3...
   ✅ Encont